In [27]:
from json import loads, JSONDecodeError
from os.path import exists
from re import match
from requests import get
from urllib.parse import urlparse

import pandas as pd

# Obter dados da API
- [Link do repositório](https://github.com/fredrike/googlescholar-api)

In [28]:
def show_user_info(user_id):
	endpoint = f'http://cse.bth.se/~fer/googlescholar-api/googlescholar.php?user={user_id}'
	page = get(endpoint)

	if not page.ok:
		raise Exception('Erro ao completar requisição para API')

	try:
		result = loads(page.text)
		return result
		
	except JSONDecodeError as json_err:
		print(json_err)
		raise Exception('Erro ao transformar conteúdo de página para dicionário')

	except Exception as err:
		print(err)
		raise Exception('Erro ao obter dados da API')
	
	# finally:
	# 	print(page.text)

# Obter id a partir do link de uma página de pesquisador

In [16]:
def get_user_id_from_profile(profile_url):
	result = urlparse(profile_url)
	if not result.query:
		raise ValueError('Sem query')

	query = result.query
	pattern = "user=[0-9a-zA-Z]+"
	has_user_id = match(pattern, query)

	if not has_user_id:
		raise ValueError('Sem id do usuário')
	
	user_id = has_user_id.group().split('=')[1]
	return user_id

# Obter nº de citações do último ano

In [37]:
def get_latest_year_citation(user_info):
	_, citations = [*user_info['citations_per_year'].items()][-1]
	return citations

# Testes

In [11]:
caio = 'https://scholar.google.com/citations?user=ztd9BnsAAAAJ&hl=pt-BR' # Caio Ponte
carlos = 'https://scholar.google.com/citations?user=dxEWdrAAAAAJ&hl=pt-BR' # Carlos Caminha
vasco = 'https://scholar.google.com/citations?user=UX9GgPgAAAAJ&hl=pt-BR' # Vasco Furtado

profiles = [ caio, carlos, vasco ]

In [29]:
for profile in profiles:
	user_id = get_user_id_from_profile(profile)
	print(profile)
	print(user_id)
	user_info = show_user_info(user_id)
	# print(user_info)
	print(get_latest_year_citation(user_info))

https://scholar.google.com/citations?user=ztd9BnsAAAAJ&hl=pt-BR
Citações em 2022: 6
https://scholar.google.com/citations?user=dxEWdrAAAAAJ&hl=pt-BR
Citações em 2022: 7
https://scholar.google.com/citations?user=UX9GgPgAAAAJ&hl=pt-BR
Citações em 2022: 80


# Ler planilha e atribuir nº de citações em nova coluna

In [38]:
researchers_csv = 'sample.csv'

if not exists(researchers_csv):
	raise Exception("Planilha não existente!")

df_researchers = pd.read_csv(researchers_csv, sep=';')
df_researchers = df_researchers.dropna(axis=1)

url_column = 'Pesquisador'
citations_column = 'Citações'

df_researchers[citations_column] = df_researchers[url_column].apply(
	lambda x: 
	get_latest_year_citation(
		show_user_info(
			get_user_id_from_profile(x)
		)
	)
)
print(df_researchers)

                                         Pesquisador  Citações
0  https://scholar.google.com/citations?user=ztd9...         6
1  https://scholar.google.com/citations?user=dxEW...         7
2  https://scholar.google.com/citations?user=UX9G...        80
